# Importing Libraries, defining paths & creating functions

In [3]:
# Importing libraries and paths

import pandas as pd
import gzip
import ast
from textblob import TextBlob

# Paths

path1 = 'Datasets/steam_games.json.gz'
path2 = 'Datasets/user_reviews.json.gz'
path3 = 'Datasets/users_items.json.gz'

# Creating a function to read the paths

def read_path(file):
    with gzip.open(file, 'rt', encoding='utf-8') as myfile:
        return [ast.literal_eval(line.strip()) for line in myfile]

# Reading datasets

In [4]:
#   Let's read all the datasets

# Reading the dataset from 'steam_games.json.gz' ↓↓↓
with gzip.open(path1, 'rt', encoding='utf-8') as file:
    df_games = pd.read_json(file, lines=True)

# Reading datasets 
reviews = read_path(path2)
items = read_path(path3)

# Transforming datasets to dataframes ↓:
df_reviews = pd.DataFrame(reviews)
df_items = pd.DataFrame(items)

## ETL

We are going to extract, transform and load data from datasets.

## First, let's begin cleaning 'df_games'

In [ ]:
# Let's start with df_games

# In the next code cell, I'm going to delete specific columns that we don't need for the tasks

df_games.drop(['publisher','url','reviews_url','price','early_access','developer','specs'],axis=1,inplace=True)

# I use inplace=True because I want to change the original dataframe too

In [28]:
# Next, We drop rows where are only null values

df_games.dropna(axis=0, how='all', inplace=True)

# Also, we drop dulicates in 'id' column

df_games.drop_duplicates(subset=['id'], inplace=True)

# Now, let's drop values where there is no genres, tags, app_name and title, this is because it will only cause latency in the EDA process

df_games.dropna(subset=['genres','tags'],how='all', inplace=True)
df_games.dropna(subset=['app_name','title'],how='all', inplace=True)

# We change the type of value in 'id' column to integer
df_games['id'].fillna(0, inplace=True)
df_games['id'] = df_games['id'].astype(int)

In [14]:
# It's time to transform 'release_date' column. If it's a null value just returns 'None'
# But if it's a value, just transform to datetime type 

def datetime_change(var):

    if pd.isna(var):
        return None

    try:
        return pd.to_datetime(var)
    except ValueError:
        return None

In [ ]:
df_games["release_date"] = df_games["release_date"].apply(datetime_change)

df_games["release_year"] = df_games["release_date"].dt.year

df_games['release_date'] = df_games['release_date'].astype('int64').astype(int)

In [30]:
# Last but not least let's save it to a csv file

df_games = df_games.to_csv('Games.csv', index=False)

## Second Step: Let's clean 'df_reviews'

In [9]:
df_reviews_2 = df_reviews.explode('reviews')

df_reviews_2 = pd.concat([df_reviews_2.drop(['reviews'], axis=1), df_reviews_2['reviews'].apply(pd.Series)], axis=1)

In [20]:
df_reviews_2['posted']

0        None
0        None
0        None
1        None
1        None
         ... 
25797    None
25797    None
25798    None
25798    None
25798    None
Name: posted, Length: 59333, dtype: object

In [19]:
df_reviews_2['posted'].apply(datetime_change)

df_reviews_2['posted'] = df_reviews_2['posted'].dt.year

AttributeError: Can only use .dt accessor with datetimelike values

In [10]:
df_reviews_2.columns

Index([    'user_id',    'user_url',       'funny',      'posted',
       'last_edited',     'item_id',     'helpful',   'recommend',
            'review',             0],
      dtype='object')

In [11]:
df_reviews.columns

Index(['user_id', 'user_url', 'reviews'], dtype='object')

In [ ]:
# We are going to delete innecesary columns
df_reviews = df_reviews.explode('reviews')

df_reviews = df_reviews.drop(["reviews","last_edited","index","posted"], axis=1, inplace=True)

## Third step: Cleaning 'df_items'

In [ ]:
# We save the file again

df_items_data = df_items.to_parquet("df_items_data.parquet", index=False)



In [ ]:
# We create a sample because is too big 

df_items_sample = df_items.sample(10000)
df_items_sample.to_parquet("df_items_sample.parquet", index=False)

Why did I choose Parquet? This is related to the structure of the dataset itself.
